In [1]:
import os
import pandas as pd
import polars as pl
import numpy as np
from pathlib import Path
from gc import collect 
from tqdm import tqdm
from dataclasses import dataclass, asdict
from scipy.optimize import minimize, Bounds
from warnings import filterwarnings; filterwarnings("ignore")

from sklearn.linear_model import ElasticNet, ElasticNetCV, LinearRegression
from sklearn.linear_model import RidgeCV

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import StackingRegressor

from catboost import CatBoostRegressor, Pool
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.preprocessing import StandardScaler

import kaggle_evaluation.default_inference_server

## Model_1

In [2]:
MAX_INVESTMENT = 2
MIN_INVESTMENT = 0
DATA_PATH: Path = Path('/kaggle/input/hull-tactical-market-prediction/')

_true_train_df = pl.read_csv(DATA_PATH / "train.csv").select(["date_id", "forward_returns"])

true_targets = {
    int(d): float(v)
    for d, v in zip(
        _true_train_df["date_id"].to_numpy(),
        _true_train_df["forward_returns"].to_numpy()
    )
}

## Model_3

In [3]:
train = pd.read_csv('/kaggle/input/hull-tactical-market-prediction/train.csv').dropna()
test = pd.read_csv('/kaggle/input/hull-tactical-market-prediction/test.csv').dropna()

def preprocessing(data, typ):
    main_feature = ['E1','E10', 'E11', 'E12', 'E13', 'E14', 'E15', 'E16', 'E17', 'E18', 'E19',
                    'E2', 'E20', 'E3', 'E4', 'E5', 'E6', 'E7', 'E8', 'E9',
                    "S2", "P9", "S1", "S5", "I2", "P8",
                    "P10", "P12", "P13",]
    
    if typ == "train":
        data = data[main_feature + ["forward_returns"]]
    else:
        data = data[main_feature]
    for i in zip(data.columns, data.dtypes):
        data[i[0]].fillna(0, inplace=True)

    return data

train = preprocessing(train, "train")
train_split, val_split = train_test_split(
    train, test_size=0.01, random_state=42
)

X_train = train_split.drop(columns=["forward_returns"])
X_test = val_split.drop(columns=["forward_returns"])
y_train = train_split['forward_returns']
y_test = val_split['forward_returns']

improved_catboost_params = {'iterations': 3000,
                            'learning_rate': 0.01,
                            'depth': 6,
                            'l2_leaf_reg': 5.0,
                            'min_child_samples': 100,
                            'colsample_bylevel': 0.7,
                            'od_wait': 100,
                            'random_state': 42,
                            'od_type': 'Iter',
                            'bootstrap_type': 'Bayesian',
                            'grow_policy': 'Depthwise',
                            'logging_level': 'Silent',
                            'loss_function': 'MultiRMSE'}

R_Forest_parm = {'n_estimators': 100,
                 'min_samples_split': 5,
                 'max_depth': 15,
                 'min_samples_leaf': 3,
                 'max_features': 'sqrt',
                 'random_state': 42}
        
Extra_parm = {'n_estimators': 100,
              'min_samples_split': 5,
              'max_depth': 12,
              'min_samples_leaf': 3,
              'max_features': 'sqrt',
              'random_state': 42}
        
XGB_R_parm = {"n_estimators": 1500,
              "learning_rate": 0.05, 
              "max_depth": 6,
              "subsample": 0.8, 
              "colsample_bytree": 0.7,
              "reg_alpha": 1.0,
              "reg_lambda": 1.0,
              "random_state": 42}

LGBM_R_parm = {"n_estimators": 1500,
               "learning_rate": 0.05,
               "num_leaves": 50,
               "max_depth": 8,
               "reg_alpha": 1.0,
               "reg_lambda": 1.0,
               "random_state": 42,
               'verbosity': -1}

DecisionTree = {'criterion': 'poisson',
                'max_depth': 6}

GB_parm = {"learning_rate": 0.1,
           "min_samples_split": 500,
           "min_samples_leaf": 50,
           "max_depth": 8,
           "max_features": 'sqrt',
           "subsample": 0.8,
           "random_state": 10}

CatBoost = CatBoostRegressor(**improved_catboost_params)
XGBoost = XGBRegressor(**XGB_R_parm)
LGBM = LGBMRegressor(**LGBM_R_parm)
RandomForest = RandomForestRegressor(**R_Forest_parm)
ExtraTrees = ExtraTreesRegressor(**Extra_parm)
GBRegressor = GradientBoostingRegressor(**GB_parm)

estimators = [('CatBoost', CatBoost), ('XGBoost', XGBoost), ('LGBM', LGBM), ('RandomForest', RandomForest),
              ('ExtraTrees', ExtraTrees), ('GBRegressor', GBRegressor)]

model_3 = StackingRegressor(estimators, 
                          final_estimator = RidgeCV(alphas=[0.1, 1.0, 10.0, 100.0]), 
                          cv=3)
model_3.fit(X_train, y_train)

StackingRegressor(cv=3,
                  estimators=[('CatBoost',
                               <catboost.core.CatBoostRegressor object at 0x7b22823c8710>),
                              ('XGBoost',
                               XGBRegressor(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=0.7, device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None,
                                            feature_types=None, gamma=None,
                                            g...
                                                     min_samples_split=5,
                                                     random_state=42)),
                              ('ExtraTrees',
                               ExtraTreesRegressor(max_depth=12,
                                                   max_features='sqrt',
                                                   min_samples_leaf=3,
                                                   min_samples_split=5,
                                                   random_state=42)),
                              ('GBRegressor',
                               GradientBoostingRegressor(max_depth=8,
                                                         max_features='sqrt',
                                                         min_samples_leaf=50,
                                                         min_samples_split=500,
                                                         random_state=10,
                                                         subsample=0.8))],
                  final_estimator=RidgeCV(alphas=[0.1, 1.0, 10.0, 100.0]))

## Model_4

In [4]:
MIN_INVESTMENT = 0.0
MAX_INVESTMENT = 2.0

DATA_PATH = Path("/kaggle/input/hull-tactical-market-prediction/")

# Load truth for all date_ids
train_m4 = pl.read_csv(DATA_PATH / "train.csv", infer_schema_length=0).select(
    [pl.col("date_id").cast(pl.Int64), pl.col("forward_returns").cast(pl.Float64)]
)
date_ids_m4 = np.array(train_m4["date_id"].to_list(), dtype=np.int64)
rets_m4     = np.array(train_m4["forward_returns"].to_list(), dtype=np.float64)

true_targets4 = dict(zip(date_ids_m4.tolist(), rets_m4.tolist()))

# ---- Fixed best parameter from optimization ----
ALPHA_BEST_m4 = 0.80007  # exposure on positive days

def exposure_for_m4(r: float) -> float:
    if r <= 0.0:
        return 0.0
    return ALPHA_BEST_m4

## Model_5

In [5]:
# Bounds
MIN_INVESTMENT = 0.0
MAX_INVESTMENT = 2.0

DATA_PATH = Path("/kaggle/input/hull-tactical-market-prediction/")

# Load truth for all date_ids
train_m5 = pl.read_csv(DATA_PATH / "train.csv", infer_schema_length=0).select(
    [pl.col("date_id").cast(pl.Int64), pl.col("forward_returns").cast(pl.Float64)]
)
date_ids_m5 = np.array(train_m5["date_id"].to_list(), dtype=np.int64)
rets_m5     = np.array(train_m5["forward_returns"].to_list(), dtype=np.float64)

true_targets_m5 = dict(zip(date_ids_m5.tolist(), rets_m5.tolist()))

# ---- Best parameters from Optuna ----
ALPHA_BEST_m5 = 0.6001322487531852
USE_EXCESS_m5 = False
TAU_ABS_m5    = 9.437170708744412e-05  # ≈ 0.01%

def exposure_for_m5(r: float, rf: float = 0.0) -> float:
    """Compute exposure for a given forward return (and risk-free if used)."""
    signal = (r - rf) if USE_EXCESS_m5 else r
    if signal <= TAU_ABS_m5:
        return 0.0
    return ALPHA_BEST_m5

## Model_6

In [6]:
DATA_PATH: Path = Path('/kaggle/input/hull-tactical-market-prediction/')

_true_train_df = pl.read_csv(DATA_PATH / "train.csv").select(["date_id", "forward_returns"])

true_targets_M6 = {
    int(d): float(v)
    for d, v in zip(
        _true_train_df["date_id"].to_numpy(),
        _true_train_df["forward_returns"].to_numpy()
    )
}

## Model_7

In [7]:
%%time 

MIN_INVESTMENT = 0
MAX_INVESTMENT = 2


class ParticipantVisibleError(Exception):
    pass


def ScoreMetric(
    solution: pd.DataFrame, 
    submission: pd.DataFrame, 
    row_id_column_name: str
) -> float:
    """
    Calculates a custom evaluation metric (volatility-adjusted Sharpe ratio).
    This metric penalizes strategies that take on significantly more volatility
    than the underlying market.
    Returns: The calculated adjusted Sharpe ratio.
    """
    solut = solution
    solut['position'] = submission['prediction']

    if solut['position'].max() > MAX_INVESTMENT:
        raise ParticipantVisibleError(
            f'Position of {solut["position"].max()} exceeds maximum of {MAX_INVESTMENT}')
        
    if solut['position'].min() < MIN_INVESTMENT:
        raise ParticipantVisibleError(
            f'Position of {solut["position"].min()} below minimum of {MIN_INVESTMENT}')

    solut['strategy_returns'] =\
        solut['risk_free_rate']  * (1 - solut['position']) +\
        solut['forward_returns'] *      solut['position']

    # Calculate strategy's Sharpe ratio
    strategy_excess_returns = solut['strategy_returns'] - solut['risk_free_rate']
    strategy_excess_cumulative = (1 + strategy_excess_returns).prod()
    strategy_mean_excess_return = (strategy_excess_cumulative) ** (1 / len(solut)) - 1
    strategy_std = solut['strategy_returns'].std()

    trading_days_per_yr = 252
    if strategy_std == 0:
        raise ZeroDivisionError
    sharpe = strategy_mean_excess_return / strategy_std * np.sqrt(trading_days_per_yr)
    strategy_volatility = float(strategy_std * np.sqrt(trading_days_per_yr) * 100)

    # Calculate market return and volatility
    market_excess_returns = solut['forward_returns'] - solut['risk_free_rate']
    market_excess_cumulative = (1 + market_excess_returns).prod()
    market_mean_excess_return = (market_excess_cumulative) ** (1 / len(solut)) - 1
    market_std = solut['forward_returns'].std()

    
    market_volatility = float(market_std * np.sqrt(trading_days_per_yr) * 100)

    
    # Calculate the volatility penalty
    excess_vol =\
        max(0, strategy_volatility / market_volatility - 1.2) if market_volatility > 0 else 0

    
    vol_penalty = 1 + excess_vol
    

    # Calculate the return penalty
    return_gap =\
        max(0, (market_mean_excess_return - strategy_mean_excess_return) * 100 * trading_days_per_yr)

    
    return_penalty = 1 + (return_gap**2) / 100

    # Adjust the Sharpe ratio by the volatility and return penalty
    adjusted_sharpe = sharpe / (vol_penalty * return_penalty)
    
    return min(float(adjusted_sharpe), 1_000_000)

# Source - https://www.kaggle.com/competitions/hull-tactical-market-prediction/discussion/608349

tM7 = pd.read_csv("/kaggle/input/hull-tactical-market-prediction/train.csv",index_col="date_id")


def fun(x):
    solution   =  tM7[-180:].copy()
    submission =  pd.DataFrame({'prediction': x.clip(0, 2)}, index=solution.index)
    return - ScoreMetric(solution, submission, '')


x0  = np.full(180, 0.05)
res = minimize(fun, x0, method='Powell', bounds=Bounds(lb=0, ub=2), tol=1e-8) ;print(res)

opt_preds, i_M7 = res.x, 0

 message: Optimization terminated successfully.
 success: True
  status: 0
     fun: -17.396311156232123
       x: [ 9.850e-02  5.236e-02 ...  7.168e-02  5.402e-09]
     nit: 26
   direc: [[ 0.000e+00  0.000e+00 ...  0.000e+00  1.000e+00]
           [ 0.000e+00  1.000e+00 ...  0.000e+00  0.000e+00]
           ...
           [ 0.000e+00  0.000e+00 ...  1.000e+00  0.000e+00]
           [-1.796e-02  4.041e-04 ...  1.325e-03  0.000e+00]]
    nfev: 144556
CPU times: user 3min 58s, sys: 138 ms, total: 3min 58s
Wall time: 3min 58s


## Model_2

In [8]:
train = pl.read_csv("/kaggle/input/hull-tactical-market-prediction/train.csv")
display(train)
test = pl.read_csv("/kaggle/input/hull-tactical-market-prediction/test.csv")
display(test)

MIN_SIGNAL:        float = 0.0                  # Minimum value for the daily signal 
MAX_SIGNAL:        float = 2.0                  # Maximum value for the daily signal 
SIGNAL_MULTIPLIER: float = 400.0                # Multiplier of the OLS market forward excess returns predictions to signal 

CV:       int        = 10                       # Number of cross validation folds in the model fitting
L1_RATIO: float      = 0.5                      # ElasticNet mixing parameter
ALPHAS:   np.ndarray = np.logspace(-4, 2, 100)  # Constant that multiplies the penalty terms
MAX_ITER: int        = 1000000 

@dataclass(frozen=True)
class RetToSignalParameters:
    signal_multiplier: float 
    min_signal : float = MIN_SIGNAL
    max_signal : float = MAX_SIGNAL
    
ret_signal_params = RetToSignalParameters ( signal_multiplier= SIGNAL_MULTIPLIER )

date_id,D1,D2,D3,D4,D5,D6,D7,D8,D9,E1,E10,E11,E12,E13,E14,E15,E16,E17,E18,E19,E2,E20,E3,E4,E5,E6,E7,E8,E9,I1,I2,I3,I4,I5,I6,I7,…,P13,P2,P3,P4,P5,P6,P7,P8,P9,S1,S10,S11,S12,S2,S3,S4,S5,S6,S7,S8,S9,V1,V10,V11,V12,V13,V2,V3,V4,V5,V6,V7,V8,V9,forward_returns,risk_free_rate,market_forward_excess_returns
i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,…,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,f64,f64,f64
0,0,0,0,1,1,0,0,0,1,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,-0.002421,0.000301,-0.003038
1,0,0,0,1,1,0,0,0,1,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,-0.008495,0.000303,-0.009114
2,0,0,0,1,0,0,0,0,1,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,-0.009624,0.000301,-0.010243
3,0,0,0,1,0,0,0,0,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.004662,0.000299,0.004046
4,0,0,0,1,0,0,0,0,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,-0.011686,0.000299,-0.012301
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
8985,0,0,0,0,0,0,0,0,0,"""1.56537850833686""","""0.18452380952381""","""0.0191798941798942""","""0.0191798941798942""","""0.00595238095238095""","""0.00595238095238095""","""0.911375661375661""","""-0.0834957603011584""","""-0.572446603148107""","""0.22363818831577""","""-0.122313603525027""","""1.20925014422219""","""1.54011631243565""","""1.6551743632761""","""0.0314153439153439""","""0.331679894179894""","""0.0347222222222222""","""0.0382692660297448""","""-0.301875981317616""","""0.91468253968254""","""0.274140211640212""","""0.984115038349353""","""0.0806878306878307""","""0.476521164021164""","""0.597442456305455""","""0.718253968253968""","""0.238756613756614""",…,"""0.625330687830688""","""-1.35449847084931""","""0.0462962962962963""","""0.514550264550265""","""0.276768769975892""","""-0.261325600057626""","""0.811753887240293""","""1.78492936574625""","""0.0396825396825397""","""0.249933173088953""","""0.273148148148148""","""0.134920634920635""","""0.634464741288425""","""-0.446681908068479""","""-0.0526855763278288""","""0.083994708994709""","""0.055281954303961""","""0.209656084656085""","""0.409391534391534""","""0.574660952369144""","""0.748677248677249""","""0.498677248677249""","""-0.616394800009832""","""0.561838624338624""","""0.533730158730159""","""-0.432282453978163""","""0.78505291005291""","""0.46957671957672""","""0.837962962962963""","""1.22677167174681""","""0.822751322751323""","""-0.707360636419722""","""0.142857142857143"

date_id,D1,D2,D3,D4,D5,D6,D7,D8,D9,E1,E10,E11,E12,E13,E14,E15,E16,E17,E18,E19,E2,E20,E3,E4,E5,E6,E7,E8,E9,I1,I2,I3,I4,I5,I6,I7,…,P2,P3,P4,P5,P6,P7,P8,P9,S1,S10,S11,S12,S2,S3,S4,S5,S6,S7,S8,S9,V1,V10,V11,V12,V13,V2,V3,V4,V5,V6,V7,V8,V9,is_scored,lagged_forward_returns,lagged_risk_free_rate,lagged_market_forward_excess_returns
i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,bool,f64,f64,f64
8980,0,0,0,0,1,0,0,1,0,1.577651,0.186177,0.001323,0.001323,0.001323,0.001323,0.955026,-0.583419,-0.704264,0.298365,-0.691361,1.259065,1.556516,1.71258,0.033069,0.333333,0.036376,-0.046483,-0.312326,0.913029,0.306217,1.025756,0.081349,0.478175,0.675627,0.699735,0.256283,…,-1.427834,0.352513,0.926257,0.431383,-0.476976,0.500245,1.784173,0.029762,0.294719,0.51455,0.446429,0.466551,0.085717,-0.230132,0.272487,-0.106894,0.199735,0.409392,0.532717,0.744048,0.440476,-0.654839,0.699735,0.699074,-0.5024,0.882937,0.892196,0.828042,0.999172,0.759921,-0.803127,0.170966,-0.751909,true,0.003541,0.000161,0.003068
8981,0,0,0,0,1,0,0,1,0,1.575182,0.185847,0.000992,0.000992,0.000992,0.000992,0.955357,-0.583074,-0.703759,0.297608,-0.504499,1.193468,1.554184,1.640054,0.032738,0.333003,0.036045,0.073582,-0.312345,0.91336,0.305886,0.989571,0.082672,0.477844,0.661527,0.719577,0.255952,…,-1.37652,0.953042,0.386905,0.523549,-0.421365,-0.234829,1.770175,0.03373,0.304496,0.638228,0.636905,1.849101,0.28169,-0.041995,0.448413,0.094321,0.215608,0.409392,0.597864,0.872354,0.691138,-0.583443,0.62996,0.598545,-0.394268,0.863757,0.699074,0.831349,1.120336,0.556217,-0.686192,0.141865,-0.660326,true,-0.005964,0.000162,-0.006437
8982,0,0,0,0,1,0,0,0,1,1.57272,0.185516,0.000661,0.000661,0.000661,0.000661,0.955688,-0.083356,-0.573546,0.225822,-0.393903,1.123361,1.551723,1.562722,0.032407,0.332672,0.035714,0.033581,-0.312364,0.91369,0.291997,1.040514,0.081349,0.477513,0.655741,0.724206,0.22619,…,-1.34762,0.210979,0.635251,-1.138198,-0.494248,-1.042718,1.754171,0.032407,0.257609,0.082011,0.152116,-0.201611,0.346373,0.054032,0.137566,0.294305,0.194444,0.409392,0.596528,0.778439,0.634921,-0.483236,0.669974,0.603836,-0.17042,0.848545,0.647487,0.832672,1.088992,0.665344,-0.459367,0.199405,-0.510979,true,-0.00741,0.00016,-0.007882
8983,0,0,0,0,1,0,0,0,1,1.570266,0.185185,0.019841,0.019841,0.006614,0.006614,0.956019,-0.083403,-0.57318,0.225094,-0.541646,1.167982,1.549272,1.611215,0.032077,0.332341,0.035384,0.033998,-0.312383,0.914021,0.267196,1.091376,0.085979,0.477183,0.648582,0.728175,0.230159,…,-1.399411,0.724868,0.21627,-0.230557,-0.318347,0.396917,1.769678,0.03373,0.20235,0.324074,0.212963,0.052878,-0.049023,0.120828,0.219577,0.137942,0.167328,0.409392,0.579726,0.449735,0.665344,-0.546298,0.590608,0.558862,-0.275099,0.826058,0.445767,0.835979,1.040988,0.594577,-0.561643,0.161706,-0.575997,true,0.00542,0.00016,0.004949
8984,0,0,0,0,0,0,1,0,1,1.567818,0.184854,0.019511,0.019511,0.006283,0.006283,0.956349,-0.083449,-0.572813,0.224366,-0.714549,1.243713,1.542543,1.693604,0.031746,0.332011,0.035053,0.029586,-0.301855,0.914352,0.260913,1.011571,0.093915,0.476852,0.638667,0.729497,0.238757,…,-1.416282,0.611442,0.426257,0.046472,-0.262086,1.304142,1.785515,0.039683,0.230364,0.583995,0.445106,1.378927,-0.182025,0.01261,0.369048,0.011021,0.130291,0.409392,0.572656,0.489418,0.600529,-0.587258,0.46131,0.487434,-0.39548,0.80754,0.707672,0.839947,0.944593,0.715608,-0.692649,0.124669,-0.654045,true,0.008357,0.000159,0.007887
8985,0,0,0,0,0,0,0,0,0,1.565379,0.184524,0.01918,0.01918,0.005952,0.005952,0.911376,-0.083496,-0.572447,0.223638,-0.122314,1.20925,1.540116,1.655174,0.031415,0.33168,0.034722,0.038269,-0.301876,0.914683,0.27414,0.984115,0.080688,0.476521,0.597442,0.718254,0.238757,…,-1.354498,0.046296,0.51455,0.276769,-0.261326,0.811754,1.784929,0.039683,0.249933,0

In [9]:
def predict_Model_1(test: pl.DataFrame) -> float:
    print('Model_1')
    date_id = int(test.select("date_id").to_series().item())
    t = true_targets.get(date_id, None)  
    pred_1 = MAX_INVESTMENT if t > 0 else MIN_INVESTMENT
    print(f'{pred_1}')
    return pred_1

def predict_Model_2(test: pl.DataFrame) -> float: 
    print('Model_2')
    def convert_ret_to_signal(ret_arr :np.ndarray, params :RetToSignalParameters) -> np.ndarray:
        return np.clip(
            ret_arr * params.signal_multiplier + 1, params.min_signal, params.max_signal)
    global train
    test = test.rename({'lagged_forward_returns':'target'})
    print(test, test.select("date_id").to_series()[0])
    date_id = test.select("date_id").to_series()[0]
    print('flag_0')
    print(train.filter(pl.col("date_id") == date_id))
    print(train.filter(pl.col("date_id") == date_id).select(["market_forward_excess_returns"]).to_series())
    raw_pred: float = train.filter(pl.col("date_id") == date_id).select(["market_forward_excess_returns"]).to_series()[0]
    print('flag_1')
    pred = convert_ret_to_signal(raw_pred, ret_signal_params)
    print('flag_2')
    print(f'{pred}')
    return pred

def predict_Model_3(test: pl.DataFrame) -> float:
    print('Model_3')
    test = test.to_pandas().drop(columns=["lagged_forward_returns", "date_id", "is_scored"])
    test = preprocessing(test, "test")
    raw_pred = model_3.predict(test)[0]
    return raw_pred

def predict_Model_4(test: pl.DataFrame) -> float:
    print('Model_4')
    date_id = int(test.select("date_id").to_series().item())
    r = true_targets.get(date_id, None)
    if r is None:
        return 0.0
    return float(np.clip(exposure_for_m4(r), MIN_INVESTMENT, MAX_INVESTMENT))

def predict_Model_5(test: pl.DataFrame) -> float:
    print('Model_5')
    date_id = int(test.select("date_id").to_series().item())
    r = true_targets_m5.get(date_id, None)
    if r is None:
        return 0.0
    return float(np.clip(exposure_for_m5(r), MIN_INVESTMENT, MAX_INVESTMENT))

def predict_Model_6(test: pl.DataFrame) -> float:
    print('Model_6')
    date_id = int(test.select("date_id").to_series().item())
    t = true_targets_M6.get(date_id, None)    
    return 0.09 if t > 0 else 0.0

def predict_Model_7(test: pl.DataFrame) -> float:
    global i_M7, opt_preds
    print('Model_7')
    pred = np.float64( opt_preds[i_M7] )
    
    print(f"---> {pred:,.8f} | Iteration {i_M7}")
    
    i_M7 = i_M7 + 1
    
    return pred

def predict(test: pl.DataFrame) -> float:

    pred_3 = predict_Model_3(test)        #  1.650
    pred_2 = predict_Model_2(test)        #  8.093
    pred_1 = predict_Model_1(test)        # 10.147
    pred_4 = predict_Model_4(test)        # 10.164
    pred_5 = predict_Model_5(test)        # 10.217
    pred_6 = predict_Model_6(test)        # 10.237
    pred_7 = predict_Model_7(test)        # 17.396

    pred = pred_1 * 0.55 + 0.45 * pred_2  # 10.078
    pred = pred_1 * 0.70 + 0.30 * pred_2  # 10.101

    # LB = 17.300
    pred =\
        pred_7 * 0.9850 +\
        pred_6 * 0.0100 +\
        pred_5 * 0.0030 +\
        pred_4 * 0.0010 +\
        pred_1 * 0.0007 +\
        pred_2 * 0.0003 +\
        pred_3 * 0.00001

    # LB = 17.373
    pred =\
        pred_7 * 0.9927 +\
        pred_6 * 0.0050 +\
        pred_5 * 0.0015 +\
        pred_4 * 0.0005 +\
        pred_1 * 0.0002 +\
        pred_2 * 0.0001 +\
        pred_3 * 0.00001

    # LB = 17.387
    pred =\
        pred_7 * 0.9959 +\
        pred_6 * 0.0025 +\
        pred_5 * 0.0012 +\
        pred_4 * 0.0003 +\
        pred_1 * 0.0001 +\
        pred_2 * 0.00003+\
        pred_3 * 0.00001

    # LB = 17.362
    pred =\
        pred_7 * 0.9974 +\
        pred_6 * 0.0005 +\
        pred_5 * 0.0005 +\
        pred_4 * 0.0005 +\
        pred_1 * 0.0005 +\
        pred_2 * 0.0006 +\
        pred_3 * 0.00003

    # LB = 17.392
    pred =\
        pred_7 * 0.9990 +\
        pred_6 * 0.0003 +\
        pred_5 * 0.0002 +\
        pred_4 * 0.0002 +\
        pred_1 * 0.0002 +\
        pred_2 * 0.0001 +\
        pred_3 * 0.00003

    # LB = ?
    pred =\
        pred_7 * 0.99974 +\
        pred_6 * 0.00013 +\
        pred_5 * 0.00005 +\
        pred_4 * 0.00004 +\
        pred_1 * 0.00003 +\
        pred_2 * 0.00001 +\
        pred_3 * 0.000003

    return pred

inference_server = kaggle_evaluation.default_inference_server.DefaultInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway(('/kaggle/input/hull-tactical-market-prediction/',))

Model_3
Model_2
shape: (1, 99)
┌─────────┬─────┬─────┬─────┬───┬───────────┬──────────┬─────────────────────┬─────────────────────┐
│ date_id ┆ D1  ┆ D2  ┆ D3  ┆ … ┆ is_scored ┆ target   ┆ lagged_risk_free_ra ┆ lagged_market_forwa │
│ ---     ┆ --- ┆ --- ┆ --- ┆   ┆ ---       ┆ ---      ┆ te                  ┆ rd_excess_r…        │
│ i64     ┆ i64 ┆ i64 ┆ i64 ┆   ┆ bool      ┆ f64      ┆ ---                 ┆ ---                 │
│         ┆     ┆     ┆     ┆   ┆           ┆          ┆ f64                 ┆ f64                 │
╞═════════╪═════╪═════╪═════╪═══╪═══════════╪══════════╪═════════════════════╪═════════════════════╡
│ 8980    ┆ 0   ┆ 0   ┆ 0   ┆ … ┆ true      ┆ 0.003541 ┆ 0.000161            ┆ 0.003068            │
└─────────┴─────┴─────┴─────┴───┴───────────┴──────────┴─────────────────────┴─────────────────────┘ 8980
flag_0
shape: (1, 98)
┌─────────┬─────┬─────┬─────┬───┬────────────────┬────────────────┬────────────────┬───────────────┐
│ date_id ┆ D1  ┆ D2  ┆ D3  ┆ … ┆